# **IMPORTS**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/tombackert/brain-tumor-mri-data/brain-tumor-mri-dataset/pituitary/pi-0080.jpg
/kaggle/input/datasets/tombackert/brain-tumor-mri-data/brain-tumor-mri-dataset/pituitary/pi-0262.jpg
/kaggle/input/datasets/tombackert/brain-tumor-mri-data/brain-tumor-mri-dataset/pituitary/pi-1749.jpg
/kaggle/input/datasets/tombackert/brain-tumor-mri-data/brain-tumor-mri-dataset/pituitary/pi-0108.jpg
/kaggle/input/datasets/tombackert/brain-tumor-mri-data/brain-tumor-mri-dataset/pituitary/pi-1680.jpg
/kaggle/input/datasets/tombackert/brain-tumor-mri-data/brain-tumor-mri-dataset/pituitary/pi-1702.jpg
/kaggle/input/datasets/tombackert/brain-tumor-mri-data/brain-tumor-mri-dataset/pituitary/pi-0379.jpg
/kaggle/input/datasets/tombackert/brain-tumor-mri-data/brain-tumor-mri-dataset/pituitary/pi-1420.jpg
/kaggle/input/datasets/tombackert/brain-tumor-mri-data/brain-tumor-mri-dataset/pituitary/pi-0037.jpg
/kaggle/input/datasets/tombackert/brain-tumor-mri-data/brain-tumor-mri-dataset/pituitary/pi

In [2]:
import os
import cv2
import shutil
import zipfile
from tqdm import tqdm
import random

# **Config**

In [3]:
# Defined the standard image size for models
IMG_SIZE = 256

#Train, Val, Test split ratio
SPLIT_RATIO = (0.8, 0.1, 0.1)

# Input Data path
DATASET_PATHS = [
    {"path": "/kaggle/input/datasets/tombackert/brain-tumor-mri-data", "prefix":"ds1"},
    {"path": "/kaggle/input/datasets/masoudnickparvar/brain-tumor-mri-dataset/Testing", "prefix":"ds2"},
    {"path": "/kaggle/input/datasets/masoudnickparvar/brain-tumor-mri-dataset/Training", "prefix":"ds3"}
]

# Output Path
WORKING_DIR = "/kaggle/working"
TEMP_POOL_DIR = os.path.join(WORKING_DIR, "temp_pool") # Temporary holding area
FINAL_DIR = os.path.join(WORKING_DIR, "processed_dataset") # Final Split structure

# Labels
CATEGORIES = ["glioma", "meningioma", "notumor", "pituitary"]

# **Funtions**

In [4]:
def skull_strip(img):
    """
    Removes sull and background using contour detection.
    Returns the croped brain image
    """

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None

    # Find the largest contours
    c = max(contours, key = cv2.contourArea)

    # Create mask
    mask = np.zeros(gray.shape, np.uint8)
    cv2.drawContours(mask, [c], -1, (255,255,255), -1)

    # Apply mask
    result = cv2.bitwise_and(img, img, mask=mask)

    # Crop to the brain area (remove excess black borders)
    x, y, w, h = cv2.boundingRect(c)
    result = result[y:y+h, x:x+w]
    
    return result

In [5]:
def process_and_pool_data():
    # Create Output Structure
    if os.path.exists(TEMP_POOL_DIR):
        shutil.rmtree(TEMP_POOL_DIR) # Clean start
    
    for cat in CATEGORIES:
        os.makedirs(os.path.join(TEMP_POOL_DIR, cat), exist_ok=True)

    total_images = 0
    
    # Iterate through Datasets
    for dataset in DATASET_PATHS:
        base_path = dataset["path"]
        prefix = dataset["prefix"]

        if not os.path.exists(base_path):
            print(f"⚠️ WARNING: Path not found: {base_path}")
            continue
        
        print(f"🚀 Processing Dataset: {prefix}...")
        
        # Walk through the dataset folders
        for root, dirs, files in os.walk(base_path):
            for file in tqdm(files):
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    
                    # Identify the Category
                    # We look at the parent folder name to decide the category
                    folder_name = os.path.basename(root).lower()
                    
                    target_cat = None
                    if "glioma" in folder_name: target_cat = "glioma"
                    elif "meningioma" in folder_name: target_cat = "meningioma"
                    elif "pituitary" in folder_name: target_cat = "pituitary"
                    elif "no" in folder_name: target_cat = "notumor"
                    
                    if target_cat:
                        try:
                            # Read & Skull Strip
                            img_path = os.path.join(root, file)
                            img = cv2.imread(img_path)
                            
                            stripped = skull_strip(img)
                            
                            if stripped is not None:
                                # Resize
                                resized = cv2.resize(stripped, (IMG_SIZE, IMG_SIZE))
                                
                                # Save with Unique Name
                                # Format: ds1_glioma_image24.jpg
                                new_filename = f"{prefix}_{target_cat}_{file}"
                                save_path = os.path.join(TEMP_POOL_DIR, target_cat, new_filename)
                                
                                cv2.imwrite(save_path, resized)
                                total_images += 1
                        except Exception as e:
                            print(f"Error processing {file}: {e}")

    print(f"\nProcessing Complete! Total Images: {total_images}")

In [6]:
def split_data():
    # Creating Fresh Train/Val/Test Split
    
    if os.path.exists(FINAL_DIR):
        shutil.rmtree(FINAL_DIR)
        
    # Create structure: processed_dataset / [train, val, test] / [glioma, ...]
    subsets = ["train", "val", "test"]
    for subset in subsets:
        for cat in CATEGORIES:
            os.makedirs(os.path.join(FINAL_DIR, subset, cat), exist_ok=True)
            
    # Move files
    for cat in CATEGORIES:
        source_cat_dir = os.path.join(TEMP_POOL_DIR, cat)
        if not os.path.exists(source_cat_dir): continue
        
        images = os.listdir(source_cat_dir)
        random.shuffle(images) # SHUFFLE THE DATA
        
        total = len(images)
        train_end = int(total * SPLIT_RATIO[0])
        val_end = train_end + int(total * SPLIT_RATIO[1])
        
        train_imgs = images[:train_end]
        val_imgs = images[train_end:val_end]
        test_imgs = images[val_end:]
        
        print(f"   -> {cat}: {len(train_imgs)} Train, {len(val_imgs)} Val, {len(test_imgs)} Test")
        
        for img in train_imgs:
            shutil.move(os.path.join(source_cat_dir, img), os.path.join(FINAL_DIR, "train", cat, img))
        for img in val_imgs:
            shutil.move(os.path.join(source_cat_dir, img), os.path.join(FINAL_DIR, "val", cat, img))
        for img in test_imgs:
            shutil.move(os.path.join(source_cat_dir, img), os.path.join(FINAL_DIR, "test", cat, img))
            
    # Cleanup Temp
    shutil.rmtree(TEMP_POOL_DIR)
    print("\nSplit Complete.")

In [7]:
def zip_dataset():
    print("Zipping final dataset...")
    zip_path = os.path.join(WORKING_DIR, "final_brain_tumor_dataset.zip")
    
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(FINAL_DIR):
            for file in files:
                arcname = os.path.relpath(os.path.join(root, file), WORKING_DIR)
                zipf.write(os.path.join(root, file), arcname)
    
    print(f"\nSaved to: {zip_path}")
    print("Go to the 'Output' tab of this notebook to download or create a dataset from it.")

# **Running**

In [8]:
process_and_pool_data()

🚀 Processing Dataset: ds1...


0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1621/1621 [00:21<00:00, 75.41it/s]


🚀 Processing Dataset: ds2...


0it [00:00, ?it/s]
100%|██████████| 400/400 [00:05<00:00, 77.26it/s]


🚀 Processing Dataset: ds3...


0it [00:00, ?it/s]
100%|██████████| 1400/1400 [00:16<00:00, 86.06it/s]


Processing Complete! Total Images: 14353


In [9]:
split_data()

   -> glioma: 2736 Train, 342 Val, 343 Test
   -> meningioma: 2860 Train, 357 Val, 358 Test
   -> notumor: 3040 Train, 380 Val, 380 Test
   -> pituitary: 2845 Train, 355 Val, 357 Test

Split Complete.


In [10]:
zip_dataset()

Zipping final dataset...

Saved to: /kaggle/working/final_brain_tumor_dataset.zip
Go to the 'Output' tab of this notebook to download or create a dataset from it.
